In [9]:
import findspark
findspark.init()
findspark.os.environ["PYSPARK_PYTHON"]="python3"
findspark.os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"
from pyspark.sql import SparkSession, Window
from pyspark import SparkContext, SparkConf
#import pydoop.hdfs as hdfs
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import StorageLevel

import datetime
import ujson as json
import os
from pathlib import Path
from tqdm import tqdm
import time

In [10]:
!python --version

Python 3.8.5


In [11]:
spark = SparkSession.builder.appName("panel_sample").config("PYSPARK_PYTHON","python3").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [12]:
spark

In [13]:
spark.stop()

# DECAHOSE PATTERNS

In [3]:
schm = StructType.fromJson(json.load(open("/home/nir/spark_stuff/decahose_schema.json")))
schm.add("ds", StringType(), False, None)
decahose_tweets = spark.read.json("/user/nir/decahose_tweets", schema=schm)
decahose_tweets.createOrReplaceTempView("decahose_tweets")

NameError: name 'spark' is not defined

In [170]:
!hdfs dfs -mkdir /user/etorf/decahose

In [6]:
!hdfs dfs -ls /user/etorf/decahose

Found 2 items
drwxr-xr-x   - etorf etorf          0 2021-09-30 11:47 /user/etorf/decahose/labeled_aug_to_nov
drwxr-xr-x   - etorf etorf          0 2021-09-30 11:39 /user/etorf/decahose/matched_pattern_aug_to_nov


In [62]:
import time 

#with SparkSession.builder.appName("panel_sample").config("PYSPARK_PYTHON","python3").getOrCreate() as spark:
schm = StructType.fromJson(json.load(open("/home/nir/spark_stuff/decahose_schema.json")))
schm.add("ds", StringType(), False, None)
decahose_tweets = spark.read.json("/user/nir/decahose_tweets", schema=schm)
decahose_tweets.filter("ds > '2020-07-31' AND ds <= '2020-11-31'").createOrReplaceTempView("decahose_tweets_aug_to_nov")
t0 = time.time()

after_filter = spark.sql("""
    SELECT 
              user_id, user_screen_name, user_bio, ds
    FROM
              (select text, user.id as user_id, user.screen_name as user_screen_name, user.description as user_bio, ds from decahose_tweets_aug_to_nov)
    WHERE 
               user_bio rlike '(?i)(I\\'?m|I am) a (guy\man\husband\father\dude)'
            or user_bio rlike '(?i)(I\\'?m|I am) a (girl|mother|wife|woman)'
            or user_bio rlike '(?i)wife of '
            or user_bio rlike '(?i)husband of '
            or user_bio rlike '(?i)mother (of|to) '
            or user_bio rlike '(?i)father (of|to) '
            or user_bio rlike '(?i)born in (\\\\d{4})'
            or user_bio rlike '(?i)(\\\\d{2}) years old'
            or user_bio rlike '(?i)(\\\\d{2}) y\\o (man|dude|guy)'
            or user_bio rlike '(?i)(\\\\d{2}) y\\o (woman|lady|girl)'
            or user_bio rlike '(?i)(^|, )(mother|wife|mom),'
            or user_bio rlike '(?i)(^|, )(father|dad|husband),'
""")


after_filter.write.save(
        path="/user/etorf/decahose/matched_pattern_aug_to_nov", format="json", mode="append", 
        partitionBy=["ds"],
        compression="bzip2" )


print(f"it took {time.time() - t0} seconds")

it took 4786.54461812973 seconds


In [63]:
a = spark.read.json("/user/etorf/decahose/matched_pattern_aug_to_nov")

In [65]:
a.select(["user_bio", "user_id", "user_screen_name"]).dropDuplicates().count()

916042

In [64]:
a.count()

22515409

In [74]:
#with SparkSession.builder.appName("panel_sample").config("PYSPARK_PYTHON","python3").getOrCreate() as spark:

matched = spark.read.json("/user/etorf/decahose/matched_pattern_aug_to_nov")
matched_deduped = matched.select(["user_bio", "user_id", "user_screen_name"])\
                                    .dropDuplicates().createOrReplaceTempView("matched_deduped")
t0 = time.time()

labeled = spark.sql("""
	        SELECT 
                  user_id, user_screen_name, user_bio,
                  CASE    WHEN user_bio rlike '(?i)born in (\\\\d{4})' THEN 'age_pattern_1'
                          WHEN user_bio rlike '(?i)(\\\\d{2}) years old' THEN 'age_pattern_2'
                          WHEN user_bio rlike '(?i)(\\\\d{2}) y\\o ' THEN 'age_pattern_3'
                          ELSE 'unlabeled_age'
                  END as age_pattern,
                  CASE    WHEN user_bio rlike '(?i)born in (\\\\d{4})' THEN regexp_extract(user_bio, '(?i)born in (\\\\d{4})' ,1)
                          WHEN user_bio rlike '(?i)(\\\\d{2}) years old' THEN regexp_extract(user_bio, '(?i)(\\\\d{2}) years old' ,1)
                          WHEN user_bio rlike '(?i)(\\\\d{2}) y\\o ' THEN regexp_extract(user_bio, '(?i)(\\\\d{2}) y\\o ' ,1)
                          ELSE 'unlabeled_age'
                  END as age_label,
                  CASE    WHEN user_bio rlike '(?i)(I\\'?m|I am) a (guy\man\husband\father\dude)' THEN 'gender_pattern_1'
                          WHEN user_bio rlike '(?i)(I\\'?m|I am) a (girl|mother|wife|woman)' THEN 'gender_pattern_2'
                          WHEN user_bio rlike '(?i)(\\\\d{2}) y\\o (woman|lady|girl)' THEN 'gender_pattern_3'
                          WHEN user_bio rlike '(?i)(\\\\d{2}) y\\o (man|dude|guy)' THEN 'gender_pattern_4'
                          WHEN user_bio rlike '(?i)(^|, )(mother|wife|mom),' THEN 'gender_pattern_5'
                          WHEN user_bio rlike '(?i)(^|, )(father|dad|husband),' THEN 'gender_pattern_6'
                          WHEN user_bio rlike '(?i)mother (of|to) ' THEN 'gender_pattern_7'
                          WHEN user_bio rlike '(?i)father (of|to) ' THEN 'gender_pattern_8'
                          WHEN user_bio rlike '(?i)wife of ' THEN 'gender_pattern_9'
                          WHEN user_bio rlike '(?i)husband of ' THEN 'gender_pattern_10'
                          ELSE 'unlabeled_gender'
                  END as gender_pattern,
                  CASE    WHEN user_bio rlike '(?i)(I\\'?m|I am) a (guy\man\husband\father\dude)' THEN 'MALE'
                          WHEN user_bio rlike '(?i)(I\\'?m|I am) a (girl|mother|wife|woman)' THEN 'FEMALE'
                          WHEN user_bio rlike '(?i)(\\\\d{2}) y\\o (woman|lady|girl)' THEN 'FEMALE'
                          WHEN user_bio rlike '(?i)(\\\\d{2}) y\\o (man|dude|guy)' THEN 'MALE'
                          WHEN user_bio rlike '(?i)(^|, )(mother|wife|mom),' THEN 'FEMALE'
                          WHEN user_bio rlike '(?i)(^|, )(father|dad|husband),' THEN 'MALE'
                          WHEN user_bio rlike '(?i)mother (of|to) ' THEN 'FEMALE'
                          WHEN user_bio rlike '(?i)father (of|to) ' THEN 'MALE'
                          WHEN user_bio rlike '(?i)wife of ' THEN 'FEMALE'
                          WHEN user_bio rlike '(?i)husband of ' THEN 'MALE'
                          ELSE 'unlabeled_gender'
                  END as gender_label

        FROM      matched_deduped
    """)


labeled.write.save(
            path="/user/etorf/decahose/labeled_aug_to_nov", format="json", mode="append", 
            compression="bzip2" )

print(f"it took {time.time() - t0} seconds")

it took 9.678255081176758 seconds


In [75]:
a = spark.read.json("/user/etorf/decahose/labeled_aug_to_nov")

In [76]:
a.filter("gender_label = 'FEMALE'").groupby("gender_pattern").count().show()

+----------------+------+
|  gender_pattern| count|
+----------------+------+
|gender_pattern_9| 18159|
|gender_pattern_3|  1030|
|gender_pattern_7|144019|
|gender_pattern_5|200886|
|gender_pattern_2| 22178|
+----------------+------+



In [77]:
a.filter("gender_label = 'MALE'").groupby("gender_pattern").count().show()

+-----------------+------+
|   gender_pattern| count|
+-----------------+------+
| gender_pattern_8|146582|
|gender_pattern_10|  8666|
| gender_pattern_4|   752|
| gender_pattern_6|250770|
+-----------------+------+



In [ ]:
a.filter("gender_label = 'FEMALE' AND gender_pattern='gender_pattern_7'").select('user_bio').show(40,False)

In [2]:
a = spark.read.json("/user/etorf/decahose/labeled_may_to_dec")

NameError: name 'spark' is not defined

In [41]:
a.filter("gender_label = 'FEMALE'").groupby("gender_pattern").count().show()

+----------------+-----+
|  gender_pattern|count|
+----------------+-----+
|gender_pattern_3| 1091|
|gender_pattern_7|36973|
|gender_pattern_5|    5|
|gender_pattern_2|    8|
+----------------+-----+



In [42]:
a.filter("gender_label = 'MALE'").groupby("gender_pattern").count().show()

+----------------+-----+
|  gender_pattern|count|
+----------------+-----+
|gender_pattern_4|  636|
+----------------+-----+



In [78]:
a.groupby("age_pattern").count().show()

+-------------+-------+
|  age_pattern|  count|
+-------------+-------+
|age_pattern_1|   9630|
|age_pattern_2| 139832|
|age_pattern_3|   2237|
|unlabeled_age|1680385|
+-------------+-------+



In [79]:
a.groupby("age_label").count().orderBy("age_label").show(100)

+---------+-----+
|age_label|count|
+---------+-----+
|       00|  732|
|       01|   13|
|     0121|    1|
|       02|   12|
|       03|   17|
|       04|   13|
|       05|    3|
|       06|    3|
|     0606|    1|
|       07|    8|
|       08|   16|
|       09|   13|
|       10|  856|
|     1000|    3|
|     1001|    1|
|     1005|    1|
|     1028|    1|
|     1066|    2|
|     1095|    2|
|       11|  970|
|     1111|    2|
|     1180|    1|
|       12| 1946|
|     1200|    1|
|     1215|    1|
|     1235|    2|
|     1246|    1|
|     1290|    1|
|       13| 5086|
|     1305|    1|
|     1312|    1|
|     1349|    1|
|     1353|    1|
|     1370|    1|
|     1375|    1|
|     1380|    1|
|     1392|    1|
|       14| 6712|
|     1413|    1|
|     1420|    1|
|     1421|    2|
|     1422|    1|
|     1428|    1|
|     1431|    1|
|     1438|    2|
|     1452|    1|
|     1453|    1|
|     1460|    1|
|     1474|    5|
|     1485|    6|
|     1488|    2|
|       15| 7504|
|     1500

# Age dataset

In [104]:
labled_set = spark.read.json("/user/etorf/decahose/labeled_aug_to_nov")

In [105]:
labled_set.createOrReplaceTempView("labled_set")

def validate_and_group_age(age_label):
    if age_label == 'unlabeled_age':
        return 'unlabeled_age'
    if len(age_label) == 4: # year
        year = int(age_label)
        if (year < 1923 or year > 2013):
            return None #filter out of range
        age = 2021 - year
    else: #2 גdigit age
        age = int(age_label)
        if (age < 8 or age >= 99):
            return None #filter out of range
    if age <= 18:
        return '<=18'
    if age < 30:
        return '(18-30)'
    if age < 40:
        return '[30-40)'
    else:
        return '[40-99)'
    
spark.udf.register("validate_and_group_age", validate_and_group_age, StringType())

grouped_age = spark.sql("""
    SELECT *
    FROM (
        SELECT 
            validate_and_group_age(age_label)      as age_group,
            *
        FROM 
            labled_set
        ) a
    WHERE age_group is not null and age_group <> 'unlabeled_age' 
""")

In [107]:
grouped_age.groupby("age_group").count().show()

+---------+-----+
|age_group|count|
+---------+-----+
|  [40-99)| 7282|
|  [30-40)| 8412|
|     <=18|52029|
|  (18-30)|82575|
+---------+-----+



In [96]:
grouped_age.show(4)

+---------+---------+-------------+----------------+----------------+--------------------+-------------------+----------------+
|age_group|age_label|  age_pattern|    gender_label|  gender_pattern|            user_bio|            user_id|user_screen_name|
+---------+---------+-------------+----------------+----------------+--------------------+-------------------+----------------+
|  (18-30)|       25|age_pattern_2|unlabeled_gender|unlabeled_gender|25 years old, GSU...|         1605137881|         DrPWN13|
|  (18-30)|     2002|age_pattern_1|unlabeled_gender|unlabeled_gender|-Tina, Nora, Dais...|1124801469402603520|          fefxri|
|  (18-30)|       21|age_pattern_2|unlabeled_gender|unlabeled_gender|-Italian porn act...|1207931183499300864|     Maryjane_nm|
|  (18-30)|       19|age_pattern_2|unlabeled_gender|unlabeled_gender|I Love min yoongi...| 883870766609178624|   YGeilmioamore|
+---------+---------+-------------+----------------+----------------+--------------------+--------------

In [106]:
grouped_age.select(['user_id', 'age_group', 'age_label', 'age_pattern']).orderBy(rand()).repartition(1).write.save(
            path="/user/etorf/validation_set_age_csv", format="csv"
            , header='True'
            , sep='\t'
            , mode='overwrite'
            , compression="gzip" )

In [110]:
grouped_age.groupby(["age_pattern","gender_pattern"]).count().show(50) #check overlap between datasets

+-------------+-----------------+------+
|  age_pattern|   gender_pattern| count|
+-------------+-----------------+------+
|age_pattern_1| unlabeled_gender|  8978|
|age_pattern_2| gender_pattern_9|    30|
|age_pattern_3| gender_pattern_5|     9|
|age_pattern_2| gender_pattern_4|     1|
|age_pattern_3| gender_pattern_2|    89|
|age_pattern_3| gender_pattern_8|    46|
|age_pattern_1| gender_pattern_2|    10|
|age_pattern_3| gender_pattern_4|   747|
|age_pattern_2| gender_pattern_8|   620|
|age_pattern_1| gender_pattern_8|    50|
|age_pattern_1| gender_pattern_9|     7|
|age_pattern_2|gender_pattern_10|    13|
|age_pattern_3|gender_pattern_10|     2|
|age_pattern_1| gender_pattern_4|     2|
|age_pattern_2| gender_pattern_6|    51|
|age_pattern_1| gender_pattern_6|    10|
|age_pattern_3| gender_pattern_7|    64|
|age_pattern_3| unlabeled_gender|   226|
|age_pattern_1| gender_pattern_7|    17|
|age_pattern_2| gender_pattern_5|    48|
|age_pattern_2| gender_pattern_2|   261|
|age_pattern_1| 

# Gender dataset

In [98]:
labled_set.show(2)

+-------------+-------------+------------+----------------+--------------------+-------------------+----------------+
|    age_label|  age_pattern|gender_label|  gender_pattern|            user_bio|            user_id|user_screen_name|
+-------------+-------------+------------+----------------+--------------------+-------------------+----------------+
|unlabeled_age|unlabeled_age|      FEMALE|gender_pattern_2|I am a wife, mom,...|1314042905724559362|     THEFLY20204|
|unlabeled_age|unlabeled_age|        MALE|gender_pattern_6|Husband, father, ...|           30063549|   ConstableCurt|
+-------------+-------------+------------+----------------+--------------------+-------------------+----------------+
only showing top 2 rows



In [108]:
labled_set.filter("gender_pattern <> 'unlabeled_gender'")\
            .select(['user_id', 'gender_pattern', 'gender_label']).orderBy(rand()).repartition(1).write.save(
            path="/user/etorf/validation_set_gender_csv", format="csv"
            , header='True'
            , sep='\t'
            , compression="gzip" )

In [56]:
schm = StructType.fromJson(json.load(open("/home/nir/spark_stuff/decahose_schema.json")))
schm.add("ds", StringType(), False, None)
decahose_tweets = spark.read.json("/user/nir/decahose_tweets", schema=schm)
decahose_tweets.filter("ds = '2020-06-31'").createOrReplaceTempView("decahose_sample")
t0 = time.time()

decahose_tweets.select(['text', 'retweeted_status', 'retweeted']).show(20)

+------------------------+--------------------+---------+
|                    text|    retweeted_status|retweeted|
+------------------------+--------------------+---------+
|    jaja https://t.co...|                null|    false|
|    Congratulations o...|                null|    false|
|    RT @Maximus_4EVR:...|{null, null, Mon ...|    false|
|    RT @ItsDeanBlunde...|{null, null, Mon ...|    false|
|    RT @miyaplanners:...|{null, null, Fri ...|    false|
|               @idvswe .|                null|    false|
|    RT @alqaradawy: ن...|{null, null, Sun ...|    false|
|    @starryscarlette ...|                null|    false|
|    RT @dannu34: Buen...|{null, null, Sun ...|    false|
|    RT @txemita: El G...|{null, null, Mon ...|    false|
|    RT @GoodHeartWinz...|{null, null, Sun ...|    false|
|    Mamãe vai me por ...|                null|    false|
|    RT @inlemidnight:...|{null, null, Mon ...|    false|
|    RT @PrettyGirlRaj...|{null, null, Mon ...|    false|
|    RT @Ahnon